In [1]:
import math
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import time
from torchsummary import summary
import config
from facenet_pytorch import training
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from PIL import Image
import glob
import torchvision.models as models
from util import AverageMeter, learning_rate_decay, Logger
from data.celeba import CelebA
from utils.config import create_config
from utils.collate import collate_custom

In [2]:
p = create_config("configs/env.yml", "configs/pretext/pretraining.yml", 128, 100)
p['batch_size'] = 128
p['epochs'] = 100

In [3]:
transform = transforms.Compose([
            transforms.RandomResizedCrop(**p['waugmentation_kwargs']['random_resized_crop']),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(**p['waugmentation_kwargs']['color_jitter'])
            ], p=p['waugmentation_kwargs']['color_jitter_random_apply']['p']),
            transforms.RandomGrayscale(**p['waugmentation_kwargs']['random_grayscale']),
            transforms.ToTensor(),
            transforms.Normalize(**p['waugmentation_kwargs']['normalize'])
        ])

In [4]:
val_transform = transforms.Compose([
                transforms.CenterCrop(p['augmentation_kwargs']['crop_size']),
                transforms.ToTensor(),
                transforms.Normalize(**p['augmentation_kwargs']['normalize'])
            ])

In [5]:
# Root directory for dataset
data_root = "/home/mehmetyavuz/datasets/CelebA128/"
attr_root = "/home/mehmetyavuz/datasets/list_attr_celeba.txt"
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = (128,128)
epochs = 10

In [69]:
from models.alexnet import alexnet
alexnet = alexnet()['backbone']

In [70]:
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=40, bias=True)
alexnet = alexnet.cuda()

In [8]:
folder_name = '/home/mehmetyavuz/datasets/'

In [9]:
dataset = CelebA(root=folder_name, split='train', transform=transform)
#dataset_two = torchvision.datasets.ImageFolder(root='/home/mehmetyavuz/datasets/YFCC392K/', transform=transform)
#y_YFCC = np.load('y_YFCC.npy')
#dataset_two.targets = y_YFCC
#dataset = torch.utils.data.ConcatDataset([dataset_one, dataset_two])

In [10]:
train_loader = torch.utils.data.DataLoader(dataset, num_workers=p['num_workers'], 
                batch_size=p['batch_size'], pin_memory=True, collate_fn=collate_custom,
                drop_last=True, shuffle=True)

In [11]:
attrs_default = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips", "Big_Nose", 
                 "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Chubby", "Double_Chin", "Eyeglasses", "Goatee", 
                 "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", 
                 "Oval_Face", "Pale_Skin", "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", 
                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie", "Young"]

In [12]:
dataset = CelebA(root=folder_name, split='valid', transform=val_transform)
val_loader = torch.utils.data.DataLoader(dataset, num_workers=p['num_workers'],
            batch_size=p['batch_size'], pin_memory=True, collate_fn=collate_custom,
            drop_last=False, shuffle=False)
dataset = CelebA(root=folder_name, split='test', transform=val_transform)
test_loader = torch.utils.data.DataLoader(dataset, num_workers=p['num_workers'],
            batch_size=p['batch_size'], pin_memory=True, collate_fn=collate_custom,
            drop_last=False, shuffle=False)

In [13]:
# Decide which device we want to run on
device = torch.device("cuda:0")

In [14]:
alexnet = torch.nn.DataParallel(alexnet)

In [15]:
weights = torch.load("results/CelebA/SimCLR-B128/finetuning_model.pth.tar")["model"]
for key in list(weights.keys()):
    weights[key.replace('module.backbone.', 'module.')] = weights.pop(key)
    if 'cluster_head' in key:
        del weights[key]

In [16]:
alexnet.load_state_dict(weights,strict=False)

_IncompatibleKeys(missing_keys=['module.classifier.6.weight', 'module.classifier.6.bias'], unexpected_keys=[])

In [17]:
optimizer = optim.Adam(alexnet.parameters(), lr=0.00001)
#Q = math.floor(len(train_dataset)*epochs)
scheduler = None #torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, Q)

In [71]:
loss_fn = torch.nn.BCEWithLogitsLoss()
metrics = {
    'acc': training.accuracy_ml
} 

In [99]:
for i, (name, param) in enumerate(alexnet.features.named_parameters()):
    if ('0.' in name) or ('6.' in name) or ('10.' in name):
        param.requires_grad = False
    else:
        param.requires_grad = True        

In [101]:
for name, param in alexnet..named_parameters():
    if param.requires_grad:
        print(name)

0.weight
0.bias
3.weight
3.bias
6.weight
6.bias
8.weight
8.bias
10.weight
10.bias


In [93]:
for name, param in alexnet.named_parameters():
    print(name)

features.0.weight
features.0.bias
features.3.weight
features.3.bias
features.6.weight
features.6.bias
features.8.weight
features.8.bias
features.10.weight
features.10.bias
classifier.1.weight
classifier.1.bias
classifier.4.weight
classifier.4.bias
classifier.6.weight
classifier.6.bias


In [ ]:
print('\n\nInitial')
print('-' * 10)
alexnet.eval()
training.pass_epoch(
    alexnet, loss_fn, test_loader,
    batch_metrics=metrics, show_running=True, device=device,
    #writer=writer
)

val_loss = 1
for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    alexnet.train()
    training.pass_epoch(
        alexnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        #writer=writer
    )

    alexnet.eval()
    val_metrics = training.pass_epoch(
        alexnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        #writer=writer
    )
    
    if val_metrics[0].item() < val_loss:
        val_loss = val_metrics[0].item()
        print('Test set Accuracy Lowest Validation Loss:')
        training.pass_epoch(
            alexnet, loss_fn, test_loader,
            batch_metrics=metrics, show_running=True, device=device,
            #writer=writer
        )
        torch.save(alexnet.state_dict(), "alexnet_ssl.pth")

#writer.close()